In [3]:
import numpy as np
import pandas as pd
import ast
import re
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math

In [4]:
df_megaset = pd.read_csv('../../data/megaset.csv')
df_megaset

,Name,Year,Languages,Countries,Genres,Tropes,domestic_gross,domestic_percentage,foreign_gross,foreign_percentage,worldwide_gross
0,,1916.0,"silent film, hungarian",hungary,"silent film, drama",NaN,NaN,NaN,NaN,NaN,NaN
1,!women art revolution,2010.0,english,"united states of america, canada","lgbt, history, documentary",NaN,NaN,NaN,NaN,NaN,NaN
2,#1 cheerleader camp,2010.0,NaN,united states of america,"sports, sex comedy, comedy film, comedy, teen",NaN,NaN,NaN,NaN,NaN,NaN
3,#horror,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,$,1971.0,english,united states of america,"crime fiction, heist, action/adventure, thrill...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
85115,…all the marbles,1981.0,english,united states of america,"drama, comedy-drama, comedy",NaN,NaN,NaN,NaN,NaN,NaN
85116,…and there wasn’t everafter…,NaN,english,india,short film,NaN,NaN,NaN,NaN,NaN,NaN
85117,…men olsenbanden var ikke død,1984.0,norwegian,norway,"crime fiction, comedy",NaN,NaN,NaN,NaN,NaN,NaN
85118,₤500 reward,1918.0,"silent film, english",NaN,silent film,NaN,NaN,NaN,NaN,NaN,NaN


# Genre Preference

Preferences in genre in US vs World for american movies (rank genres based on foreign revenues vs domestic revenues)

# Trope Preference

Preferences in TV tropes in US vs World for american movies (rank tropes based on foreign revenues vs domestic revenues)

# Proportion of Domestic vs Foreign gross for US movies

domestic vs foreign income, total and percentage, for american movies over time

# Budget Influence

budget vs foreign and domestic income for american movies

# Market Changes

⁠let's look for movie industry big numbers internationally - market size for countries over time, is america losing its market share

# Languages ?

More languages = more world audience? for american movies... not gonna give much if it's just american movies